# Imports & setup

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import numpy as np
import pickle
from xgboost import XGBRegressor
from PositionalModel import PositionalModel

from src.match_names import *
from config import *

In [3]:
info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating", "FPL_pos", "Price"]
to_predict = ["FPL_points"]

In [4]:
len(FEATURES_OUTFIELD)

42

# Reading from files

In [5]:
df = pd.read_csv("data/upcoming/upcoming_fixtures.csv")
CURRENT_SEASON = "2023-24"

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (2,3,6,8,36,43,44,45,59,84,86,147,208) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df.shape

(33970, 274)

In [7]:
df.columns

Index(['Unnamed: 0', 'Date', 'Day', 'Venue', 'Team', 'Opponent', 'Name',
       'Start', 'Pos', 'Min',
       ...
       'FPL_xGC_30', 'Opp_rating_30', 'Team_rating_30', 'Min_points_30',
       'Team_result_30', 'DEF', 'FWD', 'GK', 'MID', 'Finished'],
      dtype='object', length=274)

In [8]:
# df = df[df["GW"] == 2]

In [9]:
# X_test = df[df["Season"] == CURRENT_SEASON][FEATURES_OUTFIELD]
df = df[df["Season"] == CURRENT_SEASON]
# .reset_index(drop=True)

In [10]:
df.isna().sum().sort_values(ascending=False).head(20)

FPL_GW           16200
Min_points       16200
Clean_name       16200
Min_points_2     16200
Min_points_4     16200
Min_points_30    16200
Cmp%             15695
SCA              15691
Blocks           15691
T_succ           15691
T_att            15691
PrgC             15691
Carries          15691
PrgP             15691
Att              15691
Cmp              15691
GCA              15691
xA               15691
npxG             15691
xG               15691
dtype: int64

In [11]:
# df[FEATURES_OUTFIELD].iloc[0]

In [12]:
df = df[~df[FEATURES_OUTFIELD].isnull().any(axis=1)] # for some reason 45 rows are missing some feature values

In [13]:
df.shape

(16200, 274)

In [14]:
X_test = df.copy()

In [15]:
df.columns

Index(['Unnamed: 0', 'Date', 'Day', 'Venue', 'Team', 'Opponent', 'Name',
       'Start', 'Pos', 'Min',
       ...
       'FPL_xGC_30', 'Opp_rating_30', 'Team_rating_30', 'Min_points_30',
       'Team_result_30', 'DEF', 'FWD', 'GK', 'MID', 'Finished'],
      dtype='object', length=274)

In [16]:
file = open("models/GBR_positional.pkl",'rb')
model = pickle.load(file)

In [17]:
model

In [18]:
model.predict( X_test.head(5) )

[2.9040539037266537,
 2.9073119436544523,
 2.0064865956885933,
 3.6234892939927486,
 4.312367016996048]

In [19]:
model.predict( X_test.head(3) )

[2.9040539037266537, 2.9073119436544523, 2.0064865956885933]

In [20]:
model.predict( X_test.head(4) )

[2.9040539037266537,
 2.9073119436544523,
 2.0064865956885933,
 3.6234892939927486]

In [21]:
X_test.head(5)

,Unnamed: 0,Date,Day,Venue,Team,Opponent,Name,Start,Pos,Min,...,FPL_xGC_30,Opp_rating_30,Team_rating_30,Min_points_30,Team_result_30,DEF,FWD,GK,MID,Finished
76,76,2023-08-20,Sun,Home,West Ham,Chelsea,Aaron-Cresswell,0.0,0,0.0,...,1.02,1777.369116,1761.774137,NaN,1.166667,1.0,0.0,0.0,0.0,True
105,105,2023-08-13,Sun,Home,Brentford,Tottenham,Aaron-Hickey,1.0,RB,71.0,...,1.02,1773.661752,1765.611925,NaN,1.428571,1.0,0.0,0.0,0.0,True
106,106,2023-08-19,Sat,Away,Brentford,Fulham,Aaron-Hickey,1.0,RB,77.0,...,1.02,1775.117326,1767.636702,NaN,1.413793,1.0,0.0,0.0,0.0,True
181,181,2023-08-12,Sat,Home,Arsenal,Nott'ham Forest,Aaron-Ramsdale,1.0,GK,90.0,...,1.18,1780.652535,1925.740381,NaN,2.100000,0.0,0.0,1.0,0.0,True
182,182,2023-08-21,Mon,Away,Arsenal,Crystal Palace,Aaron-Ramsdale,1.0,GK,90.0,...,1.18,1770.298551,1926.720374,NaN,2.100000,0.0,0.0,1.0,0.0,True


In [22]:
X_test.head(3)

,Unnamed: 0,Date,Day,Venue,Team,Opponent,Name,Start,Pos,Min,...,FPL_xGC_30,Opp_rating_30,Team_rating_30,Min_points_30,Team_result_30,DEF,FWD,GK,MID,Finished
76,76,2023-08-20,Sun,Home,West Ham,Chelsea,Aaron-Cresswell,0.0,0,0.0,...,1.02,1777.369116,1761.774137,NaN,1.166667,1.0,0.0,0.0,0.0,True
105,105,2023-08-13,Sun,Home,Brentford,Tottenham,Aaron-Hickey,1.0,RB,71.0,...,1.02,1773.661752,1765.611925,NaN,1.428571,1.0,0.0,0.0,0.0,True
106,106,2023-08-19,Sat,Away,Brentford,Fulham,Aaron-Hickey,1.0,RB,77.0,...,1.02,1775.117326,1767.636702,NaN,1.413793,1.0,0.0,0.0,0.0,True


# Predicting

In [23]:
def adjust_for_injuries(df):
    active_players = pd.read_csv("data/misc/active_players.csv")
    injured_players = active_players[active_players["chance_of_playing_this_round"] == 0]["name"].to_list()
    df.loc[df["Name_original"].isin(injured_players),"Pred"] = 0
    return df

In [24]:
def adjust_goalkeepers(df):
    gks = pd.read_csv("data/misc/goalkeepers.csv")
    gk_names = gks["Name"].to_list()
    gk_names = [name_fbref_to_fpl(neutralize_name(n)) for n in gk_names]
    # print(gk_names)
    df.loc[(df["FPL_pos"] == "GK") & ~(df["Name_original"].apply(neutralize_name).isin(gk_names)), "Pred"] = 0
    return df

In [25]:
def get_predictions(model, df, X):
    # make predictions on the test data and glues them to the rest of the dataframe
    predictions = model.predict(X)
    
    df_predictions = df[(df["Season"] == CURRENT_SEASON)].reset_index(drop=True)
        
    print(len(predictions), df_predictions.shape)
        
    df_predictions.loc[:, "Pred"] = predictions
    preds = df_predictions[info + ["Pred"]]
    
    preds = adjust_goalkeepers(preds)
    preds = adjust_for_injuries(preds).sort_values(by=["Pred"], ascending = False)
    
    return preds

In [26]:
X_test[FEATURES_OUTFIELD].isna()

,Was_home,Rating_difference,Price,Transfers_balance,Avg_FPL_points,Min_2,Gls_2,Ast_2,xG_2,xA_2,...,Team_CS_30,Team_score_30,Opp_score_30,Team_result_30,Player_GC_30,FPL_points_30,DEF,FWD,GK,MID
76,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
105,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
106,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
181,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
182,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33965,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
33966,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
33967,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
33968,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [27]:
# X_test.shape,  model.predict(X_test).shape

In [28]:
%%time
preds = get_predictions(model, df, X_test)

16200 (16200, 274)
CPU times: total: 328 ms
Wall time: 298 ms


C:\users\user\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [29]:
# next gameweek
preds[preds["GW"] == NEXT_GAMEWEEK].head(50)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
6680,Jack Grealish,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,MID,7.4,6.616536
5179,Erling Haaland,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,FWD,14.0,6.452933
5981,Guglielmo Vicario,4.0,2023-24,Tottenham,Burnley,0.0,1829.239014,1722.952881,GK,5.0,6.108496
11492,Miguel Almirón Rejala,4.0,2023-24,Newcastle Utd,Brighton,0.0,1884.512817,1843.758179,MID,6.4,5.775667
1127,Alexis Mac Allister,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,MID,6.0,5.543674
9260,Kieran Tierney,4.0,2023-24,Arsenal,Manchester Utd,1.0,1927.032104,1865.755249,DEF,4.4,5.519723
7379,Jarrod Bowen,4.0,2023-24,West Ham,Luton,0.0,1787.825562,1602.477417,MID,7.0,5.456490
12854,Phil Foden,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,MID,7.6,5.450814
12958,Pierre-Emile Højbjerg,4.0,2023-24,Tottenham,Burnley,0.0,1829.239014,1722.952881,MID,5.4,5.411916
2803,Callum Wilson,4.0,2023-24,Newcastle Utd,Brighton,0.0,1884.512817,1843.758179,FWD,7.9,5.392492


In [30]:
preds[(preds["FPL_pos"] == "GK") & (preds["GW"] == NEXT_GAMEWEEK)].head(30)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
5981,Guglielmo Vicario,4.0,2023-24,Tottenham,Burnley,0.0,1829.239014,1722.952881,GK,5.0,6.108496
13793,Sam Johnstone,4.0,2023-24,Crystal Palace,Wolves,1.0,1758.725586,1704.537109,GK,4.5,4.876129
12154,Nick Pope,4.0,2023-24,Newcastle Utd,Brighton,0.0,1884.512817,1843.758179,GK,5.5,4.782113
4725,Ederson Santana de Moraes,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,GK,5.5,4.509014
1197,Alisson Ramses Becker,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,GK,5.5,4.435854
8323,Jordan Pickford,4.0,2023-24,Everton,Sheffield United,0.0,1691.463989,1631.764282,GK,4.5,4.335828
707,Aaron Ramsdale,4.0,2023-24,Arsenal,Manchester Utd,1.0,1927.032104,1865.755249,GK,5.0,3.757008
7414,Jason Steele,4.0,2023-24,Brighton,Newcastle Utd,1.0,1843.758179,1884.512817,GK,4.5,3.617081
12259,Norberto Murara Neto,4.0,2023-24,Bournemouth,Brentford,0.0,1660.290894,1835.573242,GK,4.5,3.114428
13445,Robert Sánchez,4.0,2023-24,Chelsea,Nott'ham Forest,1.0,1781.043579,1677.877930,GK,4.5,2.911555


In [31]:
# all preds
preds.head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
14425,Solly March,10.0,2023-24,Brighton,Fulham,1.0,1843.758179,1732.639404,MID,6.6,7.117919
14430,Solly March,15.0,2023-24,Brighton,Brentford,1.0,1843.758179,1835.573242,MID,6.6,7.117919
14443,Solly March,28.0,2023-24,Brighton,Nott'ham Forest,1.0,1843.758179,1677.877930,MID,6.6,7.117919
14436,Solly March,21.0,2023-24,Brighton,Wolves,1.0,1843.758179,1704.537109,MID,6.6,7.117919
14421,Solly March,6.0,2023-24,Brighton,Bournemouth,1.0,1843.758179,1660.290894,MID,6.6,7.117919
14434,Solly March,19.0,2023-24,Brighton,Tottenham,1.0,1843.758179,1829.239014,MID,6.6,7.117919
14431,Solly March,16.0,2023-24,Brighton,Burnley,1.0,1843.758179,1722.952881,MID,6.6,7.117919
14441,Solly March,26.0,2023-24,Brighton,Everton,1.0,1843.758179,1691.463989,MID,6.6,7.117919
14427,Solly March,12.0,2023-24,Brighton,Sheffield United,1.0,1843.758179,1631.764282,MID,6.6,7.117919
14449,Solly March,34.0,2023-24,Brighton,Chelsea,1.0,1843.758179,1781.043579,MID,6.6,7.117919


In [32]:
preds[preds["FPL_pos"] == "GK"].head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
6011,Guglielmo Vicario,34.0,2023-24,Tottenham,Manchester City,1.0,1829.239014,2083.702148,GK,5.0,6.745906
5993,Guglielmo Vicario,16.0,2023-24,Tottenham,Newcastle Utd,1.0,1829.239014,1884.512817,GK,5.0,6.745906
5984,Guglielmo Vicario,7.0,2023-24,Tottenham,Liverpool,1.0,1829.239014,1946.521606,GK,5.0,6.745906
6001,Guglielmo Vicario,24.0,2023-24,Tottenham,Brighton,1.0,1829.239014,1843.758179,GK,5.0,6.745906
5999,Guglielmo Vicario,22.0,2023-24,Tottenham,Brentford,1.0,1829.239014,1835.573242,GK,5.0,6.745906
6012,Guglielmo Vicario,35.0,2023-24,Tottenham,Arsenal,1.0,1829.239014,1927.032104,GK,5.0,6.745906
209,Guglielmo Vicario,2.0,2023-24,Tottenham,Manchester Utd,1.0,1817.656738,1877.037842,GK,5.0,6.685888
6014,Guglielmo Vicario,37.0,2023-24,Tottenham,Burnley,1.0,1829.239014,1722.952881,GK,5.0,6.552662
6004,Guglielmo Vicario,27.0,2023-24,Tottenham,Crystal Palace,1.0,1829.239014,1758.725586,GK,5.0,6.552662
5997,Guglielmo Vicario,20.0,2023-24,Tottenham,Bournemouth,1.0,1829.239014,1660.290894,GK,5.0,6.552662


In [33]:
# points all season
pd.DataFrame(preds.groupby(["Name_original"])["Pred"].sum().sort_values(ascending=False)).head(30)

,Pred
Name_original,
Guglielmo Vicario,233.798200
Jack Grealish,222.385187
Solly March,219.113791
Erling Haaland,207.872704
Miguel Almirón Rejala,207.119125
Bruno Borges Fernandes,204.665279
Marcus Rashford,198.613715
Callum Wilson,196.944065
Phil Foden,192.038152


# Saving to file

In [34]:
preds.to_csv("predictions/preds_next_season.csv", index=False)